# Currencies.jl Tutorial

This is a brief tutorial on how and why to use the [Currencies.jl](https://github.com/TotalVerb/Currencies.jl) library. Topics to be discussed:

 - Motivation
 - Basics
 - Exchange
 - Baskets
 - Custom currencies
 - Performance notes
 
## Motivation

It is usually a bad idea to store monetary values using floating point numbers, unless one makes sure to round properly after each step, or one is certain that exact comparisons will not be required.

In [1]:
price = 1.10

1.1

In [2]:
price += 0.10

1.2000000000000002

In [3]:
price == 1.20

false

In [4]:
price ≈ 1.20

true

## Basics

The Currencies.jl library provides a fixed-point decimal type and a convenient interface for using it.

In [5]:
using Currencies

@usingcurrencies USD, EUR;

In [6]:
price = 1.10USD

1.10 USD

In [7]:
price += 0.10USD

1.20 USD

In [8]:
price == 1.20USD

true

In [9]:
typeof(price)

Currencies.Monetary{:USD,Int64,2}

In [10]:
price += 0.10EUR

LoadError: LoadError: MethodError: no method matching +(::Currencies.Monetary{:USD,Int64,2}, ::Currencies.Monetary{:EUR,Int64,2})
Closest candidates are:
  +(::Any, ::Any, !Matched::Any, !Matched::Any...)
  +{T,U,V}(::Currencies.Monetary{T,U,V}, !Matched::Currencies.Monetary{T,U,V})
  +{T<:Currencies.Basket,U<:Currencies.AbstractMonetary}(!Matched::T<:Currencies.Basket, ::U<:Currencies.AbstractMonetary)
  ...
while loading In[10], in expression starting on line 1

In [11]:
price *= 5

6.00 USD

In [12]:
price *= 1.20

7.20 USD

## Exchange

The Currencies.jl library also provides built-in integration with [fixer.io](http://fixer.io/), an API to fetch the latest European Central Bank exchange rates.

In [13]:
ecbrates()

Currencies.ExchangeRateTable with 32 entries:
  :AUD => 0.6718172657037286
  :INR => 0.013310740569673074
  :USD => 0.8847208705653365
  :ILS => 0.2350618212589911
  :ZAR => 0.0611583389395144
  :MXN => 0.05087039241420708
  :HUF => 0.0032038959374599513
  :SEK => 0.1091548142730835
  :PHP => 0.018877541389009496
  :THB => 0.025184476288815573
  :BRL => 0.25109225129312507
  :CZK => 0.03699045646223274
  :JPY => 0.007949757532395262
  :CHF => 0.9092562284051645
  :TRY => 0.3134010279553717
  :KRW => 0.0007695089763222089
  :HKD => 0.11406280297932042
  :SGD => 0.655694708543702
  :NZD => 0.6059137178865729
  :NOK => 0.1086802951756817
  :BGN => 0.5112997238981491
  :HRK => 0.133738114025116
  :CAD => 0.7030865499542994
  :EUR => 1.0
  :GBP => 1.2907389480477574
  :DKK => 0.13435803729779114
  :RON => 0.2240143369175627
  :IDR => 6.704466448503329e-5
  :CNY => 0.13621936766969528
  :PLN => 0.22769707181565646
  :RUB => 0.013553128943113453
  :MYR => 0.2256419513515953

In [14]:
ecbrates()

Currencies.ExchangeRateTable with 32 entries:
  :AUD => 0.6718172657037286
  :INR => 0.013310740569673074
  :USD => 0.8847208705653365
  :ILS => 0.2350618212589911
  :ZAR => 0.0611583389395144
  :MXN => 0.05087039241420708
  :HUF => 0.0032038959374599513
  :SEK => 0.1091548142730835
  :PHP => 0.018877541389009496
  :THB => 0.025184476288815573
  :BRL => 0.25109225129312507
  :CZK => 0.03699045646223274
  :JPY => 0.007949757532395262
  :CHF => 0.9092562284051645
  :TRY => 0.3134010279553717
  :KRW => 0.0007695089763222089
  :HKD => 0.11406280297932042
  :SGD => 0.655694708543702
  :NZD => 0.6059137178865729
  :NOK => 0.1086802951756817
  :BGN => 0.5112997238981491
  :HRK => 0.133738114025116
  :CAD => 0.7030865499542994
  :EUR => 1.0
  :GBP => 1.2907389480477574
  :DKK => 0.13435803729779114
  :RON => 0.2240143369175627
  :IDR => 6.704466448503329e-5
  :CNY => 0.13621936766969528
  :PLN => 0.22769707181565646
  :RUB => 0.013553128943113453
  :MYR => 0.2256419513515953

In [15]:
valuate(ecbrates(), :USD, 100EUR)

113.03 USD

In [16]:
valuate(ecbrates(), :CAD, 500USD)

629.17 CAD

## Basket

We can even work with several currencies at the same time using baskets.

In [17]:
StaticBasket((100USD, 50EUR))

$2$-currency `Currencies.StaticBasket`:

 - $100.00\,\mathrm{USD}$
 - $50.00\,\mathrm{EUR}$

In [18]:
ans + 50EUR

$2$-currency `Currencies.StaticBasket`:

 - $100.00\,\mathrm{USD}$
 - $100.00\,\mathrm{EUR}$

In [19]:
ans - 100USD

$1$-currency `Currencies.StaticBasket`:

 - $100.00\,\mathrm{EUR}$

In [20]:
ans == 100EUR

true

In [21]:
StaticBasket(100EUR) + 8USD

$2$-currency `Currencies.StaticBasket`:

 - $8.00\,\mathrm{USD}$
 - $100.00\,\mathrm{EUR}$

In [22]:
basket = DynamicBasket(100EUR)

$1$-currency `Currencies.DynamicBasket`:

 - $100.00\,\mathrm{EUR}$

In [23]:
push!(basket, 8USD)

$2$-currency `Currencies.DynamicBasket`:

 - $8.00\,\mathrm{USD}$
 - $100.00\,\mathrm{EUR}$

In [24]:
basket

$2$-currency `Currencies.DynamicBasket`:

 - $8.00\,\mathrm{USD}$
 - $100.00\,\mathrm{EUR}$

In [25]:
for val in basket
    display(val)
end

8.00 USD

100.00 EUR

In [26]:
basket[:EUR]

100.00 EUR

## Custom Currencies

If a currency is not in ISO 4217, it is not too difficult to add it for custom use.

In [27]:
@usingcustomcurrency pts "Points" 4

1.0000 pts

In [28]:
6pts

6.0000 pts

In [29]:
push!(basket, pts)

$3$-currency `Currencies.DynamicBasket`:

 - $8.00\,\mathrm{USD}$
 - $1.0000\,\mathrm{pts}$
 - $100.00\,\mathrm{EUR}$

## Performance

Using `Monetary` objects is usually faster than using floating point numbers (while making sure to round properly), but not quite as fast as using integers. `Monetary` objects also come with higher memory overhead sometimes. These issues are being worked on and will hopefully become less severe with new releases of the library. For instance, consider the following function to make change for euros:

In [30]:
const COINS_I = [50000, 20000, 10000, 5000, 2000, 1000, 500, 200, 100, 50, 20, 10, 5, 2, 1]
const COINS = [500EUR, 200EUR, 100EUR, 50EUR, 20EUR, 10EUR, 5EUR, 2EUR, 1EUR, 0.5EUR,
    0.2EUR, 0.1EUR, 0.05EUR, 0.02EUR, 0.01EUR]

function change(amount::Int)
    coins = Dict{Int, Int}()
    for denomination in COINS_I
        coins[denomination], amount = divrem(amount, denomination)
    end
    coins
end

function change(amount::Monetary{:EUR,Int,2})
    coins = Dict{Monetary{:EUR,Int,2}, Int}()
    for denomination in COINS
        coins[denomination], amount = divrem(amount, denomination)
    end
    coins
end

change (generic function with 2 methods)

In [32]:
@time change(33333)


  0.000010 seconds (11 allocations: 2.125 KB)


Dict{Int64,Int64} with 15 entries:
  200 => 1
  100 => 1
  50 => 0
  2 => 1
  10000 => 1
  10 => 1
  5000 => 0
  1000 => 1
  2000 => 1
  20000 => 1
  500 => 0
  5 => 0
  20 => 1
  50000 => 0
  1 => 1

In [34]:
@time change(333.33EUR)

  0.000020 seconds (38 allocations: 2.547 KB)


Dict{Currencies.Monetary{:EUR,Int64,2},Int64} with 15 entries:
  500.0EUR => 0
  5.0EUR => 0
  0.1EUR => 1
  0.02EUR => 1
  20.0EUR => 1
  0.2EUR => 1
  50.0EUR => 0
  0.5EUR => 0
  0.05EUR => 0
  2.0EUR => 1
  200.0EUR => 1
  100.0EUR => 1
  0.01EUR => 1
  1.0EUR => 1
  10.0EUR => 1

In [35]:
(333.33EUR).val

33333